In [5]:
import pandas as pd
import xarray as xr

In [6]:
filepath = "../premise/data/metals/metals_db.csv"

In [7]:
df = pd.read_csv(filepath)

In [8]:
df

,metal,unit,year,mean,median,min,max,origin_var
0,Dysprosium,kg/MW,2015,2.512950,2.20,0.0516,11.6,Electro Motor
1,Neodymium,kg/MW,2015,10.388654,7.50,3.4000,36.5,Electro Motor
2,Dysprosium,kg/MW,2020,2.500000,2.10,1.9000,5.6,Electro Motor
3,Neodymium,kg/MW,2020,3.700000,3.60,3.4000,4.1,Electro Motor
4,Dysprosium,kg/MW,2030,1.433333,1.40,1.3000,1.6,Electro Motor
...,...,...,...,...,...,...,...,...
261,Nickel,kg/MW,2015,270.333333,256.00,255.0000,300.0,Nuclear
262,Silver,kg/MW,2015,8.150000,8.15,8.0000,8.3,Nuclear
263,Vanadium,kg/MW,2015,0.800000,0.80,0.6000,1.0,Nuclear
264,Yttrium,kg/MW,2015,0.750000,0.75,0.5000,1.0,Nuclear


In [9]:
df = df.melt(
    id_vars=["metal", "year", "origin_var"],
    value_vars=["mean", "median", "min", "max"]
)

In [10]:
df

,metal,year,origin_var,variable,value
0,Dysprosium,2015,Electro Motor,mean,2.512950
1,Neodymium,2015,Electro Motor,mean,10.388654
2,Dysprosium,2020,Electro Motor,mean,2.500000
3,Neodymium,2020,Electro Motor,mean,3.700000
4,Dysprosium,2030,Electro Motor,mean,1.433333
...,...,...,...,...,...
1059,Nickel,2015,Nuclear,max,300.000000
1060,Silver,2015,Nuclear,max,8.300000
1061,Vanadium,2015,Nuclear,max,1.000000
1062,Yttrium,2015,Nuclear,max,1.000000


In [11]:
array = df.groupby(["metal", "origin_var", "year", "variable"]).mean()["value"].to_xarray()

In [12]:
array = array.interpolate_na(dim="year", method="nearest", fill_value="extrapolate")
array = array.bfill(dim="year")
array = array.ffill(dim="year")

In [22]:
import numpy as np
np.isnan(array.sel(metal="Cobalt", origin_var="AEL", variable="mean", year=2020).values)

False

In [41]:
from premise import *
from datapackage import Package
import brightway2 as bw
bw.projects.set_current("new")

In [42]:
years = [
    #2005,
    #2010,
    #2015,
    #2020,
    #2025,
    2030,
    #2035,
    #2040,
    #2045,
    #2050,
    #2060,
    #2070,
    #2080,
    #2090,
    #2100,
]
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": y}
    for y in years
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        #external_scenarios=[Package(url) for url in urls]
)

premise v.(1, 4, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [43]:
ndb.update_all()
ndb.update_buses()
ndb.update_cars()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
Extracted 1 worksheets in 3.67 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!

/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Log of changes in photovoltaics efficiencies saved in /Users/romain/GitHub/premise/premise/data/logs
Create biomass markets.
Replace biomass inputs.
Create region-specific power plants.
Empty old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Log of deleted electricity markets saved in /Users/romain/GitHub/premise/premise/data/logs
Log of created electricity markets saved in /Users/romain/GitHub/

In [44]:
ndb.write_db_to_matrices()

Write new database(s) to matrix.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Matrices saved in /Users/romain/GitHub/premise/premise/data/export/image/SSP2-Base/2030.


In [7]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/premise/data/export/scenario_report.


In [10]:
from premise.scenario_report import *
from premise import *
from premise.activity_maps import InventorySet
from datapackage import Package
import brightway2 as bw
bw.projects.set_current("new")
years = [
    2005,
]
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": y}
    for y in years
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ecoinvent 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        #external_scenarios=[Package(url) for url in urls]
)

inv = InventorySet(ndb.scenarios[0]["database"])

premise v.(1, 4, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [56]:
# the directory to the set o.production_volumes files produced by premise
iam_model = "image"
scenario = "SSP2-Base"
years = [
    #2005,
    #2010,
    #2015,
    #2020,
    #2025,
    2030,
    #2035,
    #2040,
    #2045,
    #2050,
    #2060,
    #2070,
    #2080,
    #2090,
    #2100,
]

elec_vars = [
    'Biomass CHP', 'Biomass CHP CCS', 'Biomass ST', 'Biomass IGCC CCS',
    'Biomass IGCC', 'Coal PC', 'Coal IGCC', 'Coal IGCC CCS', 'Coal CHP',
    'Coal CHP CCS', 'Gas OC', 'Gas CC', 'Gas CHP', 'Gas CHP CCS',
    'Gas CC CCS', 'Geothermal', 'Hydro', 'Nuclear', 'Oil ST', 'Oil CC',
    'Oil CC CCS', 'Oil CHP', 'Oil CHP CCS', 'Solar CSP',
    'Solar PV Centralized', 'Solar PV Residential', 'Wind Onshore',
    'Wind Offshore'
]

metals = [
    "Dysprosium",
    "Neodymium",
    "Nickel",
    "Manganese",
    "Yttrium",
    "Vanadium",
    "Cobalt",
    "Lithium",
    "Sulfur",
    "Platinum",
    "Cerium",
    "Lanthanum",
    "Zirconium",
    "Gadolinium",
    "Scandium",
    "Strontium",
    "Potassium",
    "Iridium",
    "Silver",
    "Germanium",
    "Indium",
    "Gallium",
    "Selenium",
    "Cadmium",
    "Tellurium",
    "Copper",
    "Manganese",
    "Nickel"
]

In [70]:
from csv import reader
import numpy as np
from scipy import sparse
import scipy.sparse.linalg as sp


results = []

for year in years:
    print(year)
    DIR = Path(f"../premise/data/export/{iam_model}/{scenario}/{year}") 

    # creates dict of activities <--> indices in A matrix
    A_inds = dict()
    with open(DIR / "A_matrix_index.csv", 'r') as read_obj:
        csv_reader = reader(read_obj, delimiter=";")
        for row in csv_reader:
            A_inds[(row[0], row[1], row[2], row[3])] = row[4]
    A_inds_rev = {int(v):k for k, v in A_inds.items()}

    # creates dict of bio flow <--> indices in B matrix
    B_inds = dict()
    with open(DIR / "B_matrix_index.csv", 'r') as read_obj:
        csv_reader = reader(read_obj, delimiter=";")
        for row in csv_reader:
            B_inds[(row[0], row[1], row[2], row[3])] = row[4]
    B_inds_rev = {int(v):k for k, v in B_inds.items()}

    # create a sparse A matrix
    A_coords = np.genfromtxt(DIR / "A_matrix.csv", delimiter=";", skip_header=1)
    I = A_coords[:, 0].astype(int)
    J = A_coords[:, 1].astype(int)
    A = sparse.csr_matrix((A_coords[:,2], (J, I)))

    # create a sparse B matrix
    B_coords = np.genfromtxt(DIR / "B_matrix.csv", delimiter=";", skip_header=1)
    I = B_coords[:, 0].astype(int)
    J = B_coords[:, 1].astype(int)
    B = sparse.csr_matrix((B_coords[:,2] * -1, (I, J)), shape=(A.shape[0], len(B_inds)))
    
    metals_ids = [int(B_inds[b]) for b in B_inds if any(m in b[0] for m in metals) and b[1] == "natural resource"]
    
    
    for region in ndb.scenarios[0]["iam data"].production_volumes.region.values:
        conversion_EJ_to_kWh = 2.778e+11
        elec_prod = ndb.scenarios[0]["iam data"].production_volumes.sel(variables=elec_vars, region=region, year=year).sum().values * conversion_EJ_to_kWh

        act_id = ("market group for electricity, low voltage", "electricity, low voltage", "kilowatt hour", region)
        f = np.float64(np.zeros(A.shape[0]))
        f[int(A_inds[act_id])] = elec_prod
        A_inv = sp.spsolve(A, f)
        C = A_inv * B
        results.append([year, region, "market group for electricity, low voltage", elec_prod] + C[metals_ids].tolist())

BRA
CAN
CEU
CHN
EAF
INDIA
INDO
JAP
KOR
ME
MEX
NAF
OCE
RCAM
RSAF
RSAM
RSAS
RUS
SAF
SEAS
STAN
TUR
UKR
USA
WAF
WEU
World


In [73]:
len(metals_ids)

66

In [76]:
import pandas as pd
pd.DataFrame(results, columns=["year", "region", "product", "", "quantity"] + [B_inds_rev[m][0] for m in metals_ids])

,year,region,product,,quantity,"Lithium, 0.15% in brine, in ground","Zirconium, in ground","Silver, Ag 7.6E-5%, Au 9.7E-5%, in ore, in ground","Tellurium, 0.5ppm in sulfide, Te 0.2ppm, Cu and Ag, in crude ore, in ground","Nickel, Ni 2.5E+0%, in mixed ore, in ground",...,"Copper, 1.13% in sulfide, Cu 0.76% and Ni 0.76% in crude ore, in ground","Potassium, in ground","Lanthanum, in ground","Silver, 3.2ppm in sulfide, Ag 1.2ppm, Cu and Te, in crude ore, in ground","Nickel, 1.13% in sulfide, Ni 0.76% and Cu 0.76% in crude ore, in ground","Platinum, Pt 4.7E-7%, in mixed ore, in ground","Silver, 0.01% in crude ore, in ground","Indium, in ground","Copper, 0.99% in sulfide, Cu 0.36% and Mo 8.2E-3% in crude ore, in ground","Copper, in ground"
0,2030,BRA,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(7.6395e+1...,0.0,2.776774e+05,0.0,0.0,0.0,...,0.0,4.879799e+05,2.018026e+04,0.0,0.0,0.0,0.0,6.535770e+04,0.0,1.488201e+08
1,2030,CAN,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(6.91722e+...,0.0,9.908795e+05,0.0,0.0,0.0,...,0.0,2.829119e+05,2.002221e+04,0.0,0.0,0.0,0.0,1.051043e+04,0.0,1.353157e+08
2,2030,CEU,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(5.9727e+1...,0.0,1.071640e+06,0.0,0.0,0.0,...,0.0,1.392196e+06,3.807564e+04,0.0,0.0,0.0,0.0,1.304121e+05,0.0,1.216194e+08
3,2030,CHN,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(1.1150892...,0.0,3.240121e+07,0.0,0.0,0.0,...,0.0,1.133868e+07,6.595050e+05,0.0,0.0,0.0,0.0,1.172539e+06,0.0,2.295213e+09
4,2030,EAF,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(1.9446e+1...,0.0,2.935643e+05,0.0,0.0,0.0,...,0.0,1.368337e+06,1.025941e+04,0.0,0.0,0.0,0.0,5.088433e+02,0.0,3.958894e+07
5,2030,INDIA,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(3.094692e...,0.0,3.248166e+06,0.0,0.0,0.0,...,0.0,1.574213e+07,2.033610e+05,0.0,0.0,0.0,0.0,9.839039e+05,0.0,6.911314e+08
6,2030,INDO,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(5.30598e+...,0.0,2.397327e+05,0.0,0.0,0.0,...,0.0,2.260850e+06,2.144746e+04,0.0,0.0,0.0,0.0,2.493462e+03,0.0,1.033953e+08
7,2030,JAP,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(1.077864e...,0.0,1.554835e+06,0.0,0.0,0.0,...,0.0,2.838894e+06,6.860531e+04,0.0,0.0,0.0,0.0,9.480537e+03,0.0,2.190952e+08
8,2030,KOR,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(7.52838e+...,0.0,1.493269e+07,0.0,0.0,0.0,...,0.0,1.198139e+06,2.844217e+04,0.0,0.0,0.0,0.0,3.850238e+03,0.0,1.432263e+08
9,2030,ME,"market group for electricity, low voltage","electricity, low voltage",<xarray.DataArray 'value' ()>\narray(1.62513e+...,0.0,2.117762e+06,0.0,0.0,0.0,...,0.0,6.872701e+05,7.680136e+04,0.0,0.0,0.0,0.0,1.108760e+05,0.0,3.155360e+08


In [9]:
for f in bw.Database("biosphere3"):
    if f["categories"] == ("natural resource", "in ground"):
        print(f["name"], ";", f["categories"][0], ";", f["categories"][1], ";", f["unit"], ";", f["code"])

Astatine, in ground ; natural resource ; in ground ; kilogram ; e58cbe2f-15da-5fbe-8899-d632e3cbdfe9
Tellurium, 0.5ppm in sulfide, Te 0.2ppm, Cu and Ag, in crude ore, in ground ; natural resource ; in ground ; kilogram ; 7a81cd45-7f4c-40b3-989c-6a65f42df999
Uranium, in ground ; natural resource ; in ground ; kilogram ; 2ba5e39b-adb6-4767-a51d-90c1cf32fe98
Gadolinium, 0.15% in bastnasite, 0.015% in crude ore, in ground ; natural resource ; in ground ; kilogram ; b878ca93-d699-421e-a4b6-f694dc627062
Sylvite, 25 % in sylvinite, in ground ; natural resource ; in ground ; kilogram ; d80610f2-df83-4e2a-9dc3-f74fced6577f
Zinc, in ground ; natural resource ; in ground ; kilogram ; be73218b-18af-492e-96e6-addd309d1e32
Pd, Pd 7.3E-4%, Pt 2.5E-4%, Rh 2.0E-5%, Ni 2.3E+0%, Cu 3.2E+0% in ore, in ground ; natural resource ; in ground ; kilogram ; 535bbc83-033b-42fe-9a68-8dc9eb420385
Praseodymium, 0.42% in bastnasite, 0.042% in crude ore, in ground ; natural resource ; in ground ; kilogram ; 909bc093-